In [ ]:
#Import the libaries

import pathlib
import pandas as pd
from google.cloud import bigquery
import numpy as np
from pathlib import Path
import itertools
from itertools import zip_longest
from collections import Counter
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
from itertools import zip_longest
from functools import reduce

### Extraction the data in Google BigQuery

In [ ]:
# ──────────────────────────────────────────────────────────────────────────────
# CONFIGURATION
# ──────────────────────────────────────────────────────────────────────────────
OUTPUT_DIR = pathlib.Path("data_export")
OUTPUT_DIR.mkdir(exist_ok=True)

# Initialize BigQuery client (uses Application Default Credentials)
client = bigquery.Client(project="datathon-aki-mimic")

# ──────────────────────────────────────────────────────────────────────────────
# 1) DIAGNOSES_ICD
# ──────────────────────────────────────────────────────────────────────────────
print("Querying diagnoses_icd...")
diag_query = """
SELECT
  subject_id,
  hadm_id,
  COUNT(*)                                         AS n_diag_codes,
  ARRAY_AGG(seq_num     IGNORE NULLS ORDER BY seq_num)     AS diag_seq_nums,
  ARRAY_AGG(icd_code    IGNORE NULLS ORDER BY seq_num)     AS diag_icd_codes,
  ARRAY_AGG(icd_version IGNORE NULLS ORDER BY seq_num)     AS diag_icd_versions,
  MAX(CASE WHEN icd_code LIKE 'N17%' THEN 1 ELSE 0 END)    AS outcome_aki
FROM `physionet-data.mimiciv_3_1_hosp.diagnoses_icd`
GROUP BY subject_id, hadm_id
"""
diag_df = client.query(diag_query).to_dataframe()
print(f"Retrieved {len(diag_df):,} rows from diagnoses_icd.")

# ──────────────────────────────────────────────────────────────────────────────
# 2) EMAR (medication events)
# ──────────────────────────────────────────────────────────────────────────────
print("Querying emar...")
emar_query = """
SELECT
  subject_id,
  hadm_id,
  COUNT(*)                                                      AS n_med_admin_events,
  ARRAY_AGG(medication   IGNORE NULLS ORDER BY scheduletime)    AS meds_admin_list,
  ARRAY_AGG(event_txt    IGNORE NULLS ORDER BY scheduletime)    AS med_event_texts
FROM `physionet-data.mimiciv_3_1_hosp.emar`
GROUP BY subject_id, hadm_id
"""
emar_df = client.query(emar_query).to_dataframe()
print(f"Retrieved {len(emar_df):,} rows from emar.")

# ──────────────────────────────────────────────────────────────────────────────
# 3) HCPCSEVENTS + D_HCPCS
# ──────────────────────────────────────────────────────────────────────────────
print("Querying hcpcsevents + d_hcpcs...")
hcpcs_query = """
SELECT
  he.subject_id,
  he.hadm_id,
  COUNT(*)                                                      AS n_hcpcs_events,
  ARRAY_AGG(he.hcpcs_cd           IGNORE NULLS)                  AS hcpcs_codes,
  ARRAY_AGG(he.short_description  IGNORE NULLS)                  AS hcpcs_event_desc,
  ARRAY_AGG(dh.category           IGNORE NULLS)                  AS hcpcs_cat,
  ARRAY_AGG(dh.short_description  IGNORE NULLS)                  AS hcpcs_cat_desc
FROM `physionet-data.mimiciv_3_1_hosp.hcpcsevents` AS he
JOIN `physionet-data.mimiciv_3_1_hosp.d_hcpcs`     AS dh
  ON he.hcpcs_cd = dh.code
GROUP BY he.subject_id, he.hadm_id
"""
hcpcs_df = client.query(hcpcs_query).to_dataframe()
print(f"Retrieved {len(hcpcs_df):,} rows from hcpcsevents.")

# ──────────────────────────────────────────────────────────────────────────────
# 4) PROCEDURES_ICD
# ──────────────────────────────────────────────────────────────────────────────
print("Querying procedures_icd...")
proc_query = """
SELECT
  subject_id,
  hadm_id,
  COUNT(*)                              AS n_icd_procs,
  ARRAY_AGG(icd_code IGNORE NULLS)      AS icd_proc_codes
FROM `physionet-data.mimiciv_3_1_hosp.procedures_icd`
GROUP BY subject_id, hadm_id
"""
proc_df = client.query(proc_query).to_dataframe()
print(f"Retrieved {len(proc_df):,} rows from procedures_icd.")

# ──────────────────────────────────────────────────────────────────────────────
# 5) OMR (subject-level only)
# ──────────────────────────────────────────────────────────────────────────────
print("Querying omr (patient-level)...")
omr_query = """
SELECT
  subject_id,
  COUNT(*)                            AS n_omr_events,
  ARRAY_AGG(CONCAT(result_name,':',result_value) IGNORE NULLS) AS omr_entries
FROM `physionet-data.mimiciv_3_1_hosp.omr`
GROUP BY subject_id
"""
omr_df = client.query(omr_query).to_dataframe()
print(f"Retrieved {len(omr_df):,} rows from omr.")

# ──────────────────────────────────────────────────────────────────────────────
# 6) ADMISSIONS
# ──────────────────────────────────────────────────────────────────────────────
print("Querying admissions...")
adm_query = """
SELECT
  subject_id,
  hadm_id,
  DATETIME_DIFF(dischtime, admittime, DAY) AS time_in_hospital,
  admission_type,
  admission_location,
  discharge_location,
  insurance,
  language,
  marital_status,
  race
FROM `physionet-data.mimiciv_3_1_hosp.admissions`
"""
adm_df = client.query(adm_query).to_dataframe()
print(f"Retrieved {len(adm_df):,} rows from admissions.")

# ──────────────────────────────────────────────────────────────────────────────
# 7) LABEVENTS
# ──────────────────────────────────────────────────────────────────────────────
print("Querying labevents...")
labs_query = """
SELECT
  subject_id,
  hadm_id,
  COUNT(*) AS n_lab_events
FROM `physionet-data.mimiciv_3_1_hosp.labevents`
GROUP BY subject_id, hadm_id
"""
labs_df = client.query(labs_query).to_dataframe()
print(f"Retrieved {len(labs_df):,} rows from labevents.")

# ──────────────────────────────────────────────────────────────────────────────
# 8) ICUSTAYS
# ──────────────────────────────────────────────────────────────────────────────
print("Querying icustays...")
icu_query = """
SELECT
  subject_id,
  hadm_id,
  COUNT(*) AS icu_stays,
  SUM(los) AS icu_days
FROM `physionet-data.mimiciv_3_1_icu.icustays`
GROUP BY subject_id, hadm_id
"""
icu_df = client.query(icu_query).to_dataframe()
print(f"Retrieved {len(icu_df):,} rows from icustays.")

# ──────────────────────────────────────────────────────────────────────────────
# 9) MERGE ALL DATAFRAMES IN PANDAS
# ──────────────────────────────────────────────────────────────────────────────
print("Merging tables...")
join_dfs = [diag_df, emar_df, hcpcs_df, proc_df, adm_df, labs_df, icu_df]
merged = reduce(lambda left, right: pd.merge(left, right, on=["subject_id","hadm_id"], how="left"), join_dfs)
full_df = pd.merge(merged, omr_df, on="subject_id", how="left")
print(f"Full DataFrame: {full_df.shape[0]:,} rows × {full_df.shape[1]} columns.")

# ──────────────────────────────────────────────────────────────────────────────
# 10) SAVE TO CSV & PARQUET
# ──────────────────────────────────────────────────────────────────────────────
csv_file     = OUTPUT_DIR / "dataset_risk_AKI_full.csv"
parquet_file = OUTPUT_DIR / "dataset_risk_AKI_full.parquet"
full_df.to_csv(csv_file, index=False)
print(f"Saved CSV to: {csv_file}")
full_df.to_parquet(parquet_file, index=False)
print(f"Saved Parquet to: {parquet_file}")

### Transformation data in local

In [ ]:
# ──────────────────────────────────────────────────────────────────────────────
# CONFIG — adjust paths
# ──────────────────────────────────────────────────────────────────────────────
INPUT_PARQUET = Path(
    r"C:Local_folder"
) / "dataset_risk_AKI_full.parquet"
OUTPUT_DIR = Path(
    r"C:Local_folder"
) / "exploded_chunks"
OUTPUT_DIR.mkdir(exist_ok=True)

# ──────────────────────────────────────────────────────────────────────────────
# helper to normalize any list‐like into a Python list
# ──────────────────────────────────────────────────────────────────────────────
def ensure_list(x):
    if isinstance(x, list):
        return x
    if isinstance(x, tuple):
        return list(x)
    try:
        return list(x)
    except Exception:
        return []

# ──────────────────────────────────────────────────────────────────────────────
# Columns to extract from omr_entries
# ──────────────────────────────────────────────────────────────────────────────
def extract_omr_lists(df: pd.DataFrame) -> pd.DataFrame:
    s = df['omr_entries'].astype(str)
    df['BMI_list'] = (
        s.str.findall(r'BMI \(kg/m2\):\s*([\d\.]+)')
         .apply(lambda L: [float(x) for x in L] if L else [])
    )
    df['Weight_list'] = (
        s.str.findall(r'Weight \(Lbs\):\s*(\d+)')
         .apply(lambda L: [int(x) for x in L] if L else [])
    )
    df['BP_list'] = (
        s.str.findall(r'Blood Pressure:\s*([\d/]+)')
         .apply(lambda L: L if L else [])
    )
    return df

# ──────────────────────────────────────────────────────────────────────────────
# List‐columns to explode in lock-step
# ──────────────────────────────────────────────────────────────────────────────
list_cols = [
    'diag_seq_nums',
    'diag_icd_codes',
    'diag_icd_versions',
    'meds_admin_list',
    'med_event_texts',
    'icd_proc_codes',
    'BMI_list',
    'Weight_list',
    'BP_list',
]

# ──────────────────────────────────────────────────────────────────────────────
# Process each row-group to avoid memory spikes
# ──────────────────────────────────────────────────────────────────────────────
pf = pq.ParquetFile(INPUT_PARQUET)
print(f"Total row-groups: {pf.num_row_groups}")

for rg in range(pf.num_row_groups):
    # 1) Read one row-group
    table = pf.read_row_group(rg)
    df = table.to_pandas()

    # 2) Extract BMI, Weight and Blood Pressure lists
    df = extract_omr_lists(df)

    # 3) Ensure each list-column is a Python list, preserving existing lists
    for col in list_cols:
        df[col] = df[col].apply(ensure_list)

    # 4) Combine all list-columns into one list of tuples per row
    df['_combined'] = df[list_cols].apply(
        lambda row: list(zip_longest(*[row[c] for c in list_cols], fillvalue=None)),
        axis=1
    )

    # 5) Explode the combined tuples
    before = len(df)
    df = df.explode('_combined')
    after = len(df)
    print(f"Chunk {rg+1}/{pf.num_row_groups}: rows {before:,} → {after:,}")

    # 6) Split the tuples back into separate columns
    df[list_cols] = pd.DataFrame(df['_combined'].tolist(), index=df.index)

    # 7) Drop helper and original omr_entries
    df.drop(columns=['_combined', 'omr_entries'], inplace=True)

    out_file = OUTPUT_DIR / f"exploded_chunk_{rg:03d}.parquet"
    df.to_parquet(out_file, index=False)
    print(f"  written → {out_file}")

print("All chunks processed and saved.")

In [ ]:
# Rename columns by dropping the '_list'
df.columns = df.columns.str.replace(r'_list$', '', regex=True)

# Define output path
output_path = Path(r"C:Local_folder") \
              / "dataset_risk_AKI_full_formatted.parquet"

# Save DataFrame to Parquet
df.to_parquet(output_path, index=False)

print(f"Saved formatted dataset to: {output_path}")